In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import random
import time
import re

timeDelay = random.randrange(5, 10)

In [2]:
def products(category):
    url = "https://www.influenster.com/reviews/" + category + "/"
    
    try:
        myDriver.get(url)
        myDriver.find_element_by_xpath('//*[@id="CookielawBanner"]/div/p/a').click();
    except TimeoutException:
        print("Page expired")
        
    product_link = 'https://www.influenster.com/reviews/'
    product_list = []
    click_count = 0
    while len(product_list) < 2:
        try:
            links = [a.get_attribute('href') for a in myDriver.find_elements_by_class_name('category-product')]
            for link in links:
                if link not in product_list:
                    product_list.append(link)
                    print(link)
            
            time.sleep(timeDelay)
            load_more = myDriver.find_element_by_class_name("category-products-pagination-page.next")
            load_more.click()
            click_count+=1
            time.sleep(timeDelay)
            
        except Exception as e:
            print(e)
            break
    else:
        print(click_count)
        return product_list[:1]

In [3]:
def products_cards(category):
    url = "https://www.influenster.com/reviews/" + category + "/"
    
    try:
        myDriver.get(url)
        myDriver.find_element_by_xpath('//*[@id="CookielawBanner"]/div/p/a').click();
    except TimeoutException:
        print("Page expired")
        
    product_link = 'https://www.influenster.com/reviews/'
    product_list = []
    product_detail_list = []
    click_count = 0
    while click_count < 100: #50000
        try:
            product_template = myDriver.find_element_by_class_name('category-products')
            cards = product_template.find_elements_by_class_name('category-product')
            for card in cards:
                link = card.get_attribute('href')
                if link not in product_list:
                    product_list.append(link)
                    name = card.find_element_by_class_name('category-product-title').text
                    brand = card.find_element_by_class_name('category-product-brand').text
                    star_rating = card.find_element_by_class_name('avg-stars.small').get_attribute('data-stars')
                    reviews = card.find_element_by_class_name('category-product-rating').text
                    image = card.find_element_by_css_selector('img').get_attribute('src')
                    
                    product_detail_list.append({'product_link': link, 'product_name': name, 'brand': brand, 'star_rating': star_rating, 
                                                'total_reviews': re.findall(r'[0-9.,]+', reviews)[-1], 'image_url': image})

                    print('name:', name, 'link:', link, 'brand:', brand, 'star_rating:', 
                          star_rating, 'reviews:', re.findall(r'[0-9.,]+', reviews)[-1], 'image_url:', image)
            time.sleep(timeDelay)
            load_more = product_template.find_element_by_class_name("category-products-pagination-page.next")
            load_more.click()
            click_count+=1
            time.sleep(timeDelay)
            
        except Exception as e:
            print(e)
            return product_detail_list
            break
    else:
        print(click_count)
        return product_detail_list

In [4]:
def product_details(urls):
    """Take a post url and return post details"""
    product_detail_list = []
    for link in urls:
        print(link)
        myDriver.get(link)
        time.sleep(timeDelay)
        try:    
            #prod_link = link.replace('https://www.influenster.com/reviews/','')
            prod_link = link
        except:
            prod_link=None
            
        try:
            xpath_name = '//*[@id="app-base"]/div[1]/div[2]/div[2]/div[2]/div[1]/h1'
            product_name = myDriver.find_element_by_xpath(xpath_name).text
        except Exception as e:
            print(e)
            product_name = None
            
        try:
            total_reviews = myDriver.find_element_by_xpath('//*[@id="app-base"]/div[1]/div[2]/div[2]/div[2]/div[1]/div[2]/div[2]').text
        except Exception as e:
            print(e)
            total_reviews=None
        
        try:                     
            xpath_description = '//*[@id="app-base"]/div[1]/div[2]/div[2]/div[2]/div[2]/div'
            description = myDriver.find_element_by_xpath(xpath_description).text
        except Exception as e:
            print(e)
            description=None
            
        try:
            xpath_price = '//*[@id="app-base"]/div[1]/div[2]/div[2]/div[2]/div[1]/div[4]/a/div[1]/div[1]/strong'
            price = myDriver.find_element_by_xpath(xpath_price).text
        except Exception as e:
            print(e)
            price=None
            
        product_detail_list.append({'product_link': prod_link, 'product_name': product_name,'total_reviews': total_reviews.strip("()"),'price': price, 'description': description})
        print(prod_link,product_name,total_reviews.strip("()"),price,description)
        time.sleep(timeDelay)
        
    return product_detail_list

In [5]:
def product_reviews(url):
    print(url)
    myDriver.get(url)
    time.sleep(timeDelay)
    prod_reviews_list = []
    review_text_list = []
    click_count=0

    while (1):
        try:
            cards = []
            #product_link = url.replace('https://www.influenster.com/reviews/','')
            review_wrapper = myDriver.find_element_by_class_name("search__ReviewWrapper-sc-1ylxfoz-1.hwQBqj.layoutComponents__Block-l2otzz-0.ixyxcj")
            cards = review_wrapper.find_elements_by_class_name("item.wrappers__Wrapper-sc-1mex847-0.fNRjgH")
            
            for card in cards:
                try:
                    class_review_text = 'review-text.layoutComponents__Text-l2otzz-1.kgitTG.layoutComponents__Block-l2otzz-0.ixyxcj'
                    review_text = card.find_element_by_class_name(class_review_text).text
                except Exception as e:
                    print(e)
                if review_text not in review_text_list:
                    review_text_list.append(review_text)
                    try:
                        class_review_link = 'layoutComponents__Row-l2otzz-2.hXWPE.layoutComponents__Block-l2otzz-0.ixyxcj'
                        review_link = card.find_element_by_class_name(class_review_link).find_element_by_css_selector('a').get_attribute('href')
                    except Exception as e:
                        print(e)
                        review_link = None
                    try:
                        class_review_date = 'layoutComponents__Text-l2otzz-1 dilWrZ layoutComponents__Block-l2otzz-0 ixyxcj'
                        review_date = card.find_element_by_class_name(class_review_date).text
                    except Exception as e:
                        print(e)
                        review_date = None
                    class_reviewer = 'layoutComponents__Row-l2otzz-2.MSbai.layoutComponents__Block-l2otzz-0.ixyxcj'
                    try:
                        reviewer_name = card.find_element_by_class_name(class_reviewer).find_element_by_css_selector('a').text
                    except Exception as e:
                        print(e)
                        reviewer_name = None
                    try:
                        reviewer_link = card.find_element_by_class_name(class_reviewer).find_element_by_css_selector('a').get_attribute('href')
                    except Exception as e:
                        print(e)
                        reviewer_link = None
                    try:
                        class_loc = 'author-card.authorCard__AuthorCardWrapperDesktop-sc-26la3e-1.hAEwt'
                        reviewer_loc = card.find_element_by_class_name(class_loc).find_element_by_css_selector('span').text
                    except Exception as e:
                        print(e)
                        reviewer_loc = None
                    try:
                        class_like = 'like.like__LikeWrap-sc-1om89ge-0.bnvivG'
                        likes = card.find_element_by_class_name(class_like).text
                    except Exception as e:
                        print(e)
                        likes = None
                    prod_reviews_list.append({'review_id': re.findall(r'[0-9]+', review_link), 'review_link': review_link, 'product_link': url, 'review_text': review_text,
                                              'review_date': review_date,'reviewer_name': reviewer_name, 'reviewer_loc': reviewer_loc,
                                             'likes': re.findall(r'[0-9]+', likes), 'reviewer_link': reviewer_link})
                    print('review_id: ', re.findall(r'[0-9]+', review_link), 'review_link:', review_link, 'product_link:', url, 'review_text:', review_text,
                          'review_date:', review_date, 'reviewer_name:', reviewer_name, 'reviewer_loc:', 
                          reviewer_loc, 'likes:', re.findall(r'[0-9]+', likes), 'reviewer_link:', reviewer_link)
                    
            load_more = review_wrapper.find_element_by_css_selector('li.next')
            if load_more.get_attribute("class") == 'next':
                load_more.click()
                click_count+=1
                time.sleep(timeDelay)
            else:
                break
                
        except Exception as e:
            print(e)
            break
    else:
        print('click' + click_count) 
    return prod_reviews_list

In [6]:
try:
    # set up chromedriver
    myDriver=webdriver.Chrome()
    actions = ActionChains(myDriver)
    #prod_list = products_cards("multivitamins")
    #prod_data_frame = pd.DataFrame(prod_list)
    #prod_data_frame.to_csv("influenster_product_list_multivitamins.csv", index=False)
    prod_list = ["https://www.influenster.com/reviews/meijer"]
    print(prod_list)
    #prod_details = product_details(prod_list)
    #prod_data_frame = pd.DataFrame(prod_details)
    #prod_data_frame.to_csv("influenster_product_list_dog-food.csv", index=False)
    prod_reviews = []
    for prod in prod_list:
       product_reviews_res = product_reviews(prod)
       print(product_reviews_res)
       prod_reviews_df = pd.DataFrame(product_reviews_res)
       prod_reviews_df.to_csv("influenster_product_reviews_meijer_1.csv", mode='a', index=False, header=False)

except Exception as e:
    print(e)
finally:
    myDriver.quit()
    print('exit')

['https://www.influenster.com/reviews/meijer']
https://www.influenster.com/reviews/meijer
Message: no such element: Unable to locate element: {"method":"css selector","selector":".layoutComponents__Text-l2otzz-1 dilWrZ layoutComponents__Block-l2otzz-0 ixyxcj"}
  (Session info: chrome=83.0.4103.116)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"span"}
  (Session info: chrome=83.0.4103.116)

review_id:  ['34201416'] review_link: https://www.influenster.com/reviews/meijer/34201416 product_link: https://www.influenster.com/reviews/meijer review_text: We love meijer they always have good quality products. We get our dog food here the iams mini chuncks. We also get our groceries here.  review_date: None reviewer_name: Two Pigs & A Horse T. reviewer_loc: None likes: ['27'] reviewer_link: https://www.influenster.com/twopigsandahorse
Message: no such element: Unable to locate element: {"method":"css selector","selector":".layoutComponents__Text-l2otzz

review_id:  ['44908958'] review_link: https://www.influenster.com/reviews/meijer/44908958 product_link: https://www.influenster.com/reviews/meijer review_text: Meijer is great! I love the quality of products they offer. We visit Meijer several times a week to pick up our essentials. Everything is reasonable priced. I love exploring their home deco section because they have so many cute things to make your home feel uniquely yours.  review_date: None reviewer_name: Regina_Siders S. reviewer_loc: Mishawaka likes: ['4'] reviewer_link: https://www.influenster.com/regina_siderss
Message: no such element: Unable to locate element: {"method":"css selector","selector":".layoutComponents__Text-l2otzz-1 dilWrZ layoutComponents__Block-l2otzz-0 ixyxcj"}
  (Session info: chrome=83.0.4103.116)

review_id:  ['43033325'] review_link: https://www.influenster.com/reviews/meijer/43033325 product_link: https://www.influenster.com/reviews/meijer review_text: Meijer is one of the best stores out there. I ca

review_id:  ['3437384'] review_link: https://www.influenster.com/reviews/meijer/3437384 product_link: https://www.influenster.com/reviews/meijer review_text: One thing I've noticed about meijers, if you can't find it else where go to meijers and you'll find it there! They often do offers where you buy 10 products and get the 11 free and are always having great prices on stuff. review_date: None reviewer_name: Megan Y. reviewer_loc: Leipsic likes: ['3'] reviewer_link: https://www.influenster.com/spifffymegan
Message: no such element: Unable to locate element: {"method":"css selector","selector":".layoutComponents__Text-l2otzz-1 dilWrZ layoutComponents__Block-l2otzz-0 ixyxcj"}
  (Session info: chrome=83.0.4103.116)

review_id:  ['911442'] review_link: https://www.influenster.com/reviews/meijer/911442 product_link: https://www.influenster.com/reviews/meijer review_text: I love Meijer. I moved to Canada years ago... but visiting Meijer in Michigan reminds me of home! We miss all of the gre

review_id:  ['911567'] review_link: https://www.influenster.com/reviews/meijer/911567 product_link: https://www.influenster.com/reviews/meijer review_text: I live in a college town and having a meijer right down the street is so handy. Also the employees who work there are extremely nice and make my day sometimes. review_date: None reviewer_name: Anna F. reviewer_loc: Chicago likes: ['2'] reviewer_link: https://www.influenster.com/annaflahaven
Message: no such element: Unable to locate element: {"method":"css selector","selector":".layoutComponents__Text-l2otzz-1 dilWrZ layoutComponents__Block-l2otzz-0 ixyxcj"}
  (Session info: chrome=83.0.4103.116)

review_id:  ['911395'] review_link: https://www.influenster.com/reviews/meijer/911395 product_link: https://www.influenster.com/reviews/meijer review_text: I love shopping at Meijer! You can get everything you need at one store! The produce is always fresh! review_date: None reviewer_name: Lindsay N. reviewer_loc: Grand Rapids, MI likes: [

review_id:  ['53634593'] review_link: https://www.influenster.com/reviews/meijer/53634593 product_link: https://www.influenster.com/reviews/meijer review_text: Meier has a lot of food so it is perfect for grocery shopping. I just wish they had more veggie meat! They also have many departments so you can shop for just about anything here, including pets, electronics, and beauty. review_date: None reviewer_name: Angela O. reviewer_loc: Sterling Hts, MI likes: ['1'] reviewer_link: https://www.influenster.com/angelao16
Message: no such element: Unable to locate element: {"method":"css selector","selector":".layoutComponents__Text-l2otzz-1 dilWrZ layoutComponents__Block-l2otzz-0 ixyxcj"}
  (Session info: chrome=83.0.4103.116)

review_id:  ['53293928'] review_link: https://www.influenster.com/reviews/meijer/53293928 product_link: https://www.influenster.com/reviews/meijer review_text: They have just about everything and they even have their own versions of brand name products that I would sa

...

review_id:  ['911296'] review_link: https://www.influenster.com/reviews/meijer/911296 product_link: https://www.influenster.com/reviews/meijer review_text: Family friendly.Top get big savings and stack up that poor lonely pantry of yours.This is definitely the place to shop review_date: None reviewer_name: marco C. reviewer_loc: Calexico, CA likes: [] reviewer_link: https://www.influenster.com/marcoc5
Message: no such element: Unable to locate element: {"method":"css selector","selector":".layoutComponents__Text-l2otzz-1 dilWrZ layoutComponents__Block-l2otzz-0 ixyxcj"}
  (Session info: chrome=83.0.4103.116)

review_id:  ['911292'] review_link: https://www.influenster.com/reviews/meijer/911292 product_link: https://www.influenster.com/reviews/meijer review_text: Good assortment of vegan and organic/natural foods. Prices are fair and they often hold a 10 for $10 sale. review_date: None reviewer_name: Shay W. reviewer_loc: Sterling Hts, MI likes: [] reviewer_link: https://www.influenster.

Message: no such element: Unable to locate element: {"method":"css selector","selector":"span"}
  (Session info: chrome=83.0.4103.116)

review_id:  ['911228'] review_link: https://www.influenster.com/reviews/meijer/911228 product_link: https://www.influenster.com/reviews/meijer review_text: Great store! Wonderful associates. We love the Meijer Brand Light String Cheese :) review_date: None reviewer_name: Jeanelle S. reviewer_loc: None likes: [] reviewer_link: https://www.influenster.com/jeanelles69ca
Message: no such element: Unable to locate element: {"method":"css selector","selector":".layoutComponents__Text-l2otzz-1 dilWrZ layoutComponents__Block-l2otzz-0 ixyxcj"}
  (Session info: chrome=83.0.4103.116)

review_id:  ['911222'] review_link: https://www.influenster.com/reviews/meijer/911222 product_link: https://www.influenster.com/reviews/meijer review_text: I love going to meijer for everything a lot of people say its expensive, i don't really think so but there is things that other

Message: no such element: Unable to locate element: {"method":"css selector","selector":".layoutComponents__Text-l2otzz-1 dilWrZ layoutComponents__Block-l2otzz-0 ixyxcj"}
  (Session info: chrome=83.0.4103.116)

review_id:  ['911131'] review_link: https://www.influenster.com/reviews/meijer/911131 product_link: https://www.influenster.com/reviews/meijer review_text: I do most of my shopping at Meijer. Great produce and they have a wide variety of almost everything. review_date: None reviewer_name: Kelley M. reviewer_loc: Stanwood, MI likes: [] reviewer_link: https://www.influenster.com/kelleymason9
Message: no such element: Unable to locate element: {"method":"css selector","selector":".layoutComponents__Text-l2otzz-1 dilWrZ layoutComponents__Block-l2otzz-0 ixyxcj"}
  (Session info: chrome=83.0.4103.116)

review_id:  ['911123'] review_link: https://www.influenster.com/reviews/meijer/911123 product_link: https://www.influenster.com/reviews/meijer review_text: I never knew store brand coul

review_id:  ['911056'] review_link: https://www.influenster.com/reviews/meijer/911056 product_link: https://www.influenster.com/reviews/meijer review_text: Meijer is wonderful. It's much nicer than Walmart. It's a bit more expensive, but you tend to avoid all of the welfare people at the first of the month. review_date: None reviewer_name: Chelsea reviewer_loc: None likes: [] reviewer_link: https://www.influenster.com/member-e87026b77
Message: no such element: Unable to locate element: {"method":"css selector","selector":".layoutComponents__Text-l2otzz-1 dilWrZ layoutComponents__Block-l2otzz-0 ixyxcj"}
  (Session info: chrome=83.0.4103.116)

review_id:  ['911052'] review_link: https://www.influenster.com/reviews/meijer/911052 product_link: https://www.influenster.com/reviews/meijer review_text: Love these stores!!! So much better than Walmart. Clean and bright to shop in and the workers are always happy and helpful. Also the mperks saves my family money, so that's huge! review_date: No

exit
